In [4]:
from transformers import AutoProcessor, AutoModelForImageTextToText
from PIL import Image
import torch

# Path to your local model
model_path = "models/Qwen2-VL-2B-Instruct"

# Choose device dynamically
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

# Load processor & model
processor = AutoProcessor.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForImageTextToText.from_pretrained(
    model_path,
    torch_dtype=dtype,
    device_map=None,   # we'll move it manually to avoid partial device mismatch
    trust_remote_code=True
).to(device)

# Load and convert image
image = Image.open("Invoice.png").convert("RGB")

# Create structured conversation
conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "Extract all table data from this invoice in JSON format."}
        ],
    }
]

# Get prompt string
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

# Prepare tensor inputs
inputs = processor(
    text=prompt,
    images=image,
    return_tensors="pt"
)

# 🔥 Move all tensors explicitly to the same device
for k, v in inputs.items():
    if isinstance(v, torch.Tensor):
        inputs[k] = v.to(device)

# Generate output
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=500)

# Decode result
result = processor.batch_decode(outputs, skip_special_tokens=True)[0]
print("\n🧾 Extracted Data:\n", result)


Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.67s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.19 GiB. GPU 0 has a total capacity of 8.00 GiB of which 0 bytes is free. Of the allocated memory 12.91 GiB is allocated by PyTorch, and 495.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [3]:
empty = (1)
print(bool(empty))

True


In [2]:
import torch, os
print(torch.__file__)
print(torch.version.cuda)
print(torch.version.__version__)


c:\Users\syeda\Ansar\Projects\model_setup\.venv\Lib\site-packages\torch\__init__.py
None
2.8.0+cpu


In [3]:
import sys
print(sys.executable)


c:\Users\syeda\Ansar\Projects\model_setup\.venv\Scripts\python.exe


In [2]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)


False
None


In [1]:
import torch

In [2]:
print(torch.cuda.is_available())

False
